# **Theoretical Aspects**
Main text 

# Part 0 - Download and Install Robosample

---



Robosample binaries are already build on Google colab and uploaded on Github. Clone the Github repository that contains Robosample binaries.

In [ ]:
%cd /content/
! if [ -d Notebooks ]; then echo "Notebooks exists"; else git clone --recursive https://github.com/spirilaurentiu/Notebooks.git; fi
!ls Notebooks
%cd Notebooks


/content
Cloning into 'Notebooks'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 51 (delta 8), reused 44 (delta 4), pack-reused 0
Unpacking objects: 100% (51/51), done.
2but			 libOpenMM.so		libSimTKsimbody.so
GMOLMODEL_robo		 libSimTKcommon.so	libSimTKsimbody.so.3.8
inp.2but		 libSimTKcommon.so.3.8	README.md
libGMOLMODEL_dynamic.so  libSimTKmath.so	temp
libOpenMMCUDA.so	 libSimTKmath.so.3.8	test.ipynb
libOpenMMOpenCL.so	 libSimTKmolmodel.so
/content/Notebooks


 Now we have to make sure that Robosample binary finds its libraries. Set the linker path.
 Robosample relies heavly on GPUs. Set cuda directories.

In [ ]:
!if echo $LD_LIBRARY_PATH | grep Notebook; then echo "LD_LIBRARY_PATH set";else echo "Run one of the below commands"; fi
#%env LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/Notebooks/
#!echo "$LD_LIBRARY_PATH"

!if echo $CUDA_ROOT | grep usr; then echo "CUDA directories set"; else echo "Run one of the below commands"; fi
#%env CUDA_INC_DIR=/usr/local/cuda
#%env CUDA_ROOT=/usr/local/cuda


/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/Notebooks/
LD_LIBRARY_PATH set
/usr/local/cuda
CUDA directories set


If everything went okay so far, Robosample should run on a test small molecule.

In [ ]:
!./GMOLMODEL_robo	inp.2but

Reading input...
Dumping input file:
  BOOST_MDSTEPS : 1 
  BOOST_TEMPERATURE : 600 
  DIHEDRAL : 0 1 2 3 
  DISTANCE : 0 1 
  FFSCALE : AMBER 
  FIXMAN_POTENTIAL : TRUE 
  FIXMAN_TORQUE : TRUE 
  FLEXFILE : ligand.td.flex 
  GBSA : 1.0 
  GEOMETRY : FALSE 
  INPCRD : ligand.inpcrd 
  MDSTEPS : 10 
  MOLECULES : 2but 
  NMA_OPTION : 0 
  NONBONDED_CUTOFF : 2 
  NONBONDED_METHOD : 1 
  OPENMM : TRUE 
  OPENMM_CalcOnlyNonbonded : TRUE 
  OUTPUT_DIR : temp 
  PRINT_FREQ : 1 
  PRMTOP : ligand.prmtop 
  RANDOM_WORLD_ORDER : FALSE 
  RBFILE : ligand.rb 
  REPRODUCIBLE : FALSE 
  ROOTS : 0 
  ROOT_MOBILITY : Free 
  ROUNDS : 10 
  ROUNDS_TILL_REBLOCK : 10 
  RUN_TYPE : Normal 
  SAMPLER : VV 
  SAMPLES_PER_ROUND : 1 
  SEED : 42 
  TEMPERATURE_FIN : 300 
  TEMPERATURE_INI : 300 
  THERMOSTAT : Andersen 
  THREADS : 0 
  TIMESTEPS : 0.003 
  VISUAL : FALSE 
  WORLDS : R1 
  WRITEPDBS : 0 
Done.
Molecule directory: 2but
tcmalloc: large alloc 1073741824 bytes == 0x557122554000 @  0x7f43f744c887

# Install Robosample from scratch.

Clone the Github repository.



In [ ]:
# Install prerequisites
!apt-get install cmake graphviz gfortran libglfw3-dev freeglut3-dev libglew-dev libxmu-dev libeigen3-dev doxygen subversion libblas-dev liblapack-dev libboost-all-dev swig ocl-icd-opencl-dev fftw2 libxmu-dev libxi-dev

%cd /content/
! if [ -d Roobsample ]; then echo "Notebooks exists"; else git clone --recursive https://github.com/spirilaurentiu/Robosample.git; fi

# Attach heads
%cd /content/Robosample/openmm/
!git checkout master

%cd /content/Robosample/Molmodel/Simbody01/
!git checkout master

%cd /content/Robosample/Molmodel/
!git checkout openmmCom

%cd /content/Robosample/
!git checkout trex

# Set CMAKE_MINIMUM_REQUIRED to 3.0 everywhere


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libblas-dev is already the newest version (3.7.1-4ubuntu1).
libblas-dev set to manually installed.
liblapack-dev is already the newest version (3.7.1-4ubuntu1).
libxi-dev is already the newest version (2:1.7.9-1).
libxi-dev set to manually installed.
libxmu-dev is already the newest version (2:1.1.2-2).
libxmu-dev set to manually installed.
ocl-icd-opencl-dev is already the newest version (2.2.11-1ubuntu1).
ocl-icd-opencl-dev set to manually installed.
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
graphviz is already the newest version (2.40.1-2).
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
gfortran is already the newest version (4:7.4.0-1ubuntu2.3).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt a

Run the installer.

In [ ]:
!bash build.sh -b release --cc gcc --cpp g++

-- Found PythonInterp: /usr/local/bin/python (found version "2.7.17") 
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/gcc
-- Check for working C compiler: /usr/bin/gcc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/g++
-- Check for working CXX compiler: /usr/bin/g++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Deprecation Warning at CMakeLists.txt:25 (CMAKE_POLICY):
  The OLD behavior for policy CMP0042 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explains that the OLD behaviors of all
  policies are deprecated and that a policy should be set to OLD only under
  specific short-term circumstances